# Utilización de pandas  en el análisis de datos

## poder subir un archivo, en este caso .cvs ubicado en un archivo local, Le hacemos una primera mirada de los primeros y los últimos datos para ver como está compuesta, vemos información importante y si hay duplicados.


In [23]:
import pandas as pd
from collections import defaultdict


nombre_archivo = "Coffe_sales.csv" 

try:
    df = pd.read_csv(nombre_archivo)
    
    print("¡Archivo cargado exitosamente en JupyterLite!")
    print("\nPrimeras 5 filas del DataFrame:")
    print(df.head())
    print(df.info())
    df.describe()
    print(df.duplicated().sum())
    

except Exception as e:
    print(f"Ocurrió un error al leer el archivo: {e}")





¡Archivo cargado exitosamente en JupyterLite!

Primeras 5 filas del DataFrame:
   hour_of_day cash_type  money    coffee_name Time_of_Day Weekday Month_name  \
0           10      card   38.7          Latte     Morning     Fri        Mar   
1           12      card   38.7  Hot Chocolate   Afternoon     Fri        Mar   
2           12      card   38.7  Hot Chocolate   Afternoon     Fri        Mar   
3           13      card   28.9      Americano   Afternoon     Fri        Mar   
4           13      card   38.7          Latte   Afternoon     Fri        Mar   

   Weekdaysort  Monthsort        Date             Time  
0            5          3  2024-03-01  10:15:50.520000  
1            5          3  2024-03-01  12:19:22.539000  
2            5          3  2024-03-01  12:20:18.089000  
3            5          3  2024-03-01  13:46:33.006000  
4            5          3  2024-03-01  13:48:14.626000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3547 entries, 0 to 3546
Data columns (tota

# Se le hizo un análisis al conjunto, no tiene datos faltantes o extraños, tenemos problemas con el formato de los tiempos

In [24]:

df['Full_Timestamp_Str'] = df['Date'].astype(str) + ' ' + df['Time'].astype(str)

# Para cuantificar errores
df['Transaction_DateTime'] = pd.to_datetime(df['Full_Timestamp_Str'], 
                                             errors='coerce', 
                                             format='mixed')

# Validar la limpieza y eliminar la columna temporal
nan_count = df['Transaction_DateTime'].isna().sum()
print("\n--- ¡ERROR DE TIEMPO CORREGIDO Y VALIDADO! ---")
print(f"Total de valores que fallaron en la conversión (NaT): {nan_count}")

# Limpieza del DataFrame
df = df.drop(columns=['Full_Timestamp_Str'])

# Validar la nueva estructura del DataFrame
print("\nValidación de las primeras 5 filas con el nuevo campo de tiempo:")
print(df[['Date', 'Time', 'Transaction_DateTime']].head())
df.info()

# Iterar sobre las columnas de tipo 'object' (categóricas) para ver los valores únicos
print("\n--- Inspección de Valores Únicos en Columnas Categóricas ---")
for col in df.select_dtypes(include=['object']).columns:
    print(f"\nColumna: {col}")
    
    # Muestra los valores únicos y su frecuencia de aparición
    print(df[col].value_counts())
    



--- ¡ERROR DE TIEMPO CORREGIDO Y VALIDADO! ---
Total de valores que fallaron en la conversión (NaT): 0

Validación de las primeras 5 filas con el nuevo campo de tiempo:
         Date             Time    Transaction_DateTime
0  2024-03-01  10:15:50.520000 2024-03-01 10:15:50.520
1  2024-03-01  12:19:22.539000 2024-03-01 12:19:22.539
2  2024-03-01  12:20:18.089000 2024-03-01 12:20:18.089
3  2024-03-01  13:46:33.006000 2024-03-01 13:46:33.006
4  2024-03-01  13:48:14.626000 2024-03-01 13:48:14.626
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3547 entries, 0 to 3546
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hour_of_day           3547 non-null   int64         
 1   cash_type             3547 non-null   object        
 2   money                 3547 non-null   float64       
 3   coffee_name           3547 non-null   object        
 4   Time_of_Day           3547 non-null   ob

# Ya los datos están totalmente limpios y completos

# Ahora vamos a extraer información valiosa con ayuda de pandas

In [25]:

dias_ordenados = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

demanda_cruzada = pd.crosstab(
    df['Weekday'], 
    df['Time_of_Day']
).reindex(dias_ordenados)

print("\n--- Demanda Semanal por Momento del Día (Conteo de Ventas) ---")
print(demanda_cruzada)

# Agrupamos por Momento del Día y por Nombre del Café, y contamos las ventas
ventas_por_momento_y_cafe = df.groupby(['Time_of_Day', 'coffee_name']).size()

# Usamos 'idxmax' en el nivel interior (coffee_name) para encontrar el índice (nombre del café)
# que tuvo la mayor venta en cada Time_of_Day.
cafe_lider_por_momento = ventas_por_momento_y_cafe.groupby(level=0).idxmax()

print("\n--- Café Líder de Ventas en Cada Momento del Día ---")
print(cafe_lider_por_momento)

# Inicializar y generar el diccionario (Repetimos la lógica del paso anterior para asegurar la ejecución)
reportes_por_cafe = defaultdict(pd.DataFrame)

for cafe in df['coffee_name'].unique():
    # Filtrar el DataFrame original
    df_filtrado = df[df['coffee_name'] == cafe]
    
    # Crear la Tabla Dinámica (Pivot Table)
    pivot_cafe = df_filtrado.pivot_table(
        index='Time_of_Day', 
        columns='Weekday', 
        values='hour_of_day', 
        aggfunc='count'
    )
    
    # Reordenar las columnas y rellenar los posibles valores NaN (cero ventas) con 0
    pivot_cafe = pivot_cafe.reindex(columns=dias_ordenados, fill_value=0)
    
    reportes_por_cafe[cafe] = pivot_cafe


# 1. Bucle de Impresión de Todos los Reportes (La Solicitud Clave)
print("\n" + "="*50)
print("     REPORTE COMPLETO DE VENTAS POR PRODUCTO Y DEMANDA")
print("="*50)

# Iterar sobre el diccionario e imprimir cada DataFrame de la Tabla Dinámica
for nombre_cafe, reporte_df in reportes_por_cafe.items():
    print(f"\n--- ☕ REPORTE: {nombre_cafe} ---")
    print(reporte_df)
    print("-" * 50)




--- Demanda Semanal por Momento del Día (Conteo de Ventas) ---
Time_of_Day  Afternoon  Morning  Night
Weekday                               
Mon                177      193    174
Tue                160      207    205
Wed                165      165    170
Thu                169      146    195
Fri                172      193    167
Sat                194      157    119
Sun                168      120    131

--- Café Líder de Ventas en Cada Momento del Día ---
Time_of_Day
Afternoon                (Afternoon, Latte)
Morning      (Morning, Americano with Milk)
Night                        (Night, Latte)
dtype: object

     REPORTE COMPLETO DE VENTAS POR PRODUCTO Y DEMANDA

--- ☕ REPORTE: Latte ---
Weekday      Mon  Tue  Wed  Thu  Fri  Sat  Sun
Time_of_Day                                   
Afternoon     48   33   37   35   38   45   34
Morning       39   40   33   26   32   22   23
Night         51   46   38   47   37   28   25
--------------------------------------------------

--- 

# Para sacar métricas seguras y poder ser certeros con los datos, debemos ver que los datos por días sean equitativos. Puede que hayan más lunes que jueves, llevando a lunes a tener una mayor percepción, que podría ser falsa.

In [26]:
# 1. Ventas Totales por Día: Contar el número de transacciones por cada día de la semana
ventas_totales_por_dia = df.groupby('Weekday').size().reindex(dias_ordenados, fill_value=0)

# 2. Conteo de Ocurrencias del Día (El paso clave para la normalización):
# Eliminamos las filas duplicadas por 'Date' y luego contamos cuántas veces aparece cada 'Weekday'.
# Esto nos dice cuántos Lunes, Martes, etc., hubo en el período total.
conteo_dias_unicos = df.drop_duplicates(subset=['Date'])['Weekday'].value_counts().reindex(dias_ordenados, fill_value=0)

# 3. Calcular la Tasa Promedio de Ventas por Día (Métrica Segura)
# Tasa Promedio = Ventas Totales / Conteo de Días Únicos
tasa_promedio_ventas = (ventas_totales_por_dia / conteo_dias_unicos).round(2).sort_values(ascending=False)

print("\n--- 📈 TASA PROMEDIO DE VENTAS DIARIAS NORMALIZADA ---")
print("Este es el verdadero indicador de demanda, libre de sesgo.")
print("-" * 50)
print(tasa_promedio_ventas)
print("-" * 50)

# Opcional: Mostrar el conteo de días únicos para verificar la normalización
print("\nConteo de Días Únicos en el Período Analizado (Para Verificación):")
print(conteo_dias_unicos)


--- 📈 TASA PROMEDIO DE VENTAS DIARIAS NORMALIZADA ---
Este es el verdadero indicador de demanda, libre de sesgo.
--------------------------------------------------
Weekday
Tue    10.40
Mon     9.89
Wed     9.62
Fri     9.50
Thu     9.44
Sat     8.55
Sun     7.76
dtype: float64
--------------------------------------------------

Conteo de Días Únicos en el Período Analizado (Para Verificación):
Weekday
Mon    55
Tue    55
Wed    52
Thu    54
Fri    56
Sat    55
Sun    54
Name: count, dtype: int64


## Vamos a establecer cúal fue el rango

In [29]:
df['Date'] = pd.to_datetime(df['Date'])

# Calculamos la fecha más antigua y la más reciente
fecha_inicio = df['Date'].min().strftime('%Y-%m-%d')
fecha_fin = df['Date'].max().strftime('%Y-%m-%d')

print(f"**🗓️ Rango de Fechas del Dataset:**")
print(f"Fecha de Inicio: **{fecha_inicio}**")
print(f"Fecha de Fin: **{fecha_fin}**")

**🗓️ Rango de Fechas del Dataset:**
Fecha de Inicio: **2024-03-01**
Fecha de Fin: **2025-03-23**


In [30]:


df['Date'] = pd.to_datetime(df['Date'])
fechas_presentes = df['Date'].dt.date.unique() 

# Definir el rango completo esperado
fecha_inicio = fechas_presentes.min()
fecha_fin = fechas_presentes.max()
rango_completo = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
fechas_esperadas = rango_completo.date

# Identificar las fechas faltantes
fechas_presentes_set = set(fechas_presentes)
fechas_faltantes = [fecha for fecha in fechas_esperadas if fecha not in fechas_presentes_set]

print(f"**⏳ Verificación de la Continuidad Temporal:**\n")

if fechas_faltantes:
    print(f"**¡ATENCIÓN! Se encontraron {len(fechas_faltantes)} días faltantes (saltos) en las ventas.**")
    print(f"\nPrimeros 10 días sin datos:")
    # Muestra solo la fecha en formato YYYY-MM-DD
    for fecha in fechas_faltantes[:10]:
        print(f"  - {fecha.strftime('%Y-%m-%d')}")
else:
    print("**✅ ¡Continuidad perfecta! No se encontraron días sin ventas entre la fecha de inicio y la fecha de fin.**")

**⏳ Verificación de la Continuidad Temporal:**

**¡ATENCIÓN! Se encontraron 7 días faltantes (saltos) en las ventas.**

Primeros 10 días sin datos:
  - 2024-05-01
  - 2024-05-04
  - 2024-05-05
  - 2024-11-27
  - 2025-01-01
  - 2025-01-19
  - 2025-01-23


## Aquí podemos ver los días que no se atendieron, no hay información que nos diga el por qué no se atendió o si fue que ese día no se vendió

## Ahora vamos a ver el horario de atención

In [31]:
# Agrupamos por día de la semana y encontramos la hora mínima (apertura) y máxima (cierre)
horarios_atencion = df.groupby('Weekday')['hour_of_day'].agg(
    Apertura=('min'),
    Cierre=('max')
).reset_index()


# Mostramos el resultado
print("\n**⏰ Horarios de Apertura y Cierre por Día:**")
print(horarios_atencion)


**⏰ Horarios de Apertura y Cierre por Día:**
  Weekday  Apertura  Cierre
0     Fri         6      22
1     Mon         6      22
2     Sat         7      22
3     Sun         7      22
4     Thu         7      22
5     Tue         7      22
6     Wed         7      22
